In [ ]:
pip install fastapi uvicorn faiss-cpu sentence-transformers transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

In [ ]:
# Import libraries
import faiss
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Force using CPU
device = torch.device("cpu")

# Load models
embedding_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)
generator = pipeline("text-generation", model="distilgpt2", device=-1)  # -1 forces CPU usage

# Step 1: Document Database
documents = [
    "The Amazon rainforest is the largest tropical rainforest in the world, spanning across several countries in South America. It covers approximately 5.5 million square kilometers and is home to a vast variety of plant and animal species. The Amazon plays a crucial role in regulating the Earth's climate by absorbing large amounts of carbon dioxide and producing oxygen.",
    "Machine learning (ML) is a field of artificial intelligence (AI) focused on the development of algorithms that enable computers to learn from and make predictions based on data. It includes techniques like supervised learning, unsupervised learning, and reinforcement learning.",
    "The Eiffel Tower is one of the most iconic landmarks in the world, located in Paris, France. Designed by engineer Gustave Eiffel, it was constructed between 1887 and 1889 as the entrance arch for the 1889 World's Fair.",
    "Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think, learn, and problem-solve. AI technologies include machine learning, natural language processing, robotics, and computer vision.",
    "Quantum computing is an emerging field of computing that leverages the principles of quantum mechanics to process information in fundamentally new ways. Applications of quantum computing include cryptography, material science, and optimization problems."
]

# Step 2: Encode documents into embeddings
doc_embeddings = embedding_model.encode(documents, convert_to_numpy=True)

# Step 3: Create FAISS Index
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

# Function to search for relevant documents
def search_documents(query, k=2):
    query_embedding = embedding_model.encode([query], convert_to_numpy=True)
    _, I = index.search(query_embedding, k)
    return [documents[i] for i in I[0]]

# Function to generate an answer from retrieved context
def generate_answer(query):
    retrieved_docs = search_documents(query, k=2)
    context = " ".join(retrieved_docs)
    prompt = f"Question: {query}\nContext: {context}\nAnswer:"
    response = generator(prompt, max_length=200, num_return_sequences=1)
    return response[0]['generated_text']

# Step 4: Get user input and generate response
query = input("Enter your question: ")
answer = generate_answer(query)

print("\n Retrieved Context:\n", search_documents(query, k=2))
print("\n Generated Answer:\n", answer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Enter your question: tell about amazon forest


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 Retrieved Context:
 ["The Amazon rainforest is the largest tropical rainforest in the world, spanning across several countries in South America. It covers approximately 5.5 million square kilometers and is home to a vast variety of plant and animal species. The Amazon plays a crucial role in regulating the Earth's climate by absorbing large amounts of carbon dioxide and producing oxygen.", "The Eiffel Tower is one of the most iconic landmarks in the world, located in Paris, France. Designed by engineer Gustave Eiffel, it was constructed between 1887 and 1889 as the entrance arch for the 1889 World's Fair."]

 Generated Answer:
 Question: tell about amazon forest
Context: The Amazon rainforest is the largest tropical rainforest in the world, spanning across several countries in South America. It covers approximately 5.5 million square kilometers and is home to a vast variety of plant and animal species. The Amazon plays a crucial role in regulating the Earth's climate by absorbing lar